# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902048


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:39,  1.37s/it]

Rendering models:  13%|█▎        | 4/31 [00:04<00:31,  1.17s/it]

Rendering models:  23%|██▎       | 7/31 [00:06<00:26,  1.09s/it]

Rendering models:  32%|███▏      | 10/31 [00:07<00:16,  1.28it/s]

...

Rendering models:  74%|███████▍  | 23/31 [00:07<00:01,  4.13it/s]

Rendering models:  81%|████████  | 25/31 [00:08<00:01,  4.95it/s]

Rendering models:  87%|████████▋ | 27/31 [00:08<00:00,  6.22it/s]

Rendering models:  94%|█████████▎| 29/31 [00:08<00:00,  7.52it/s]

equidad1                              0.000252
Mark_Moffatt                          0.000407
kayleebug2017                         0.007320
JessieDawn                            0.002710
ZigZagoon999                          0.004423
nickoftona                            0.000820
not-logged-in-0b0736c8991a7cc2017a    0.007279
not-logged-in-8ca9191f589d7b339215    0.373311
pollux2728                            0.009912
not-logged-in-dabd20e53f02c0431022    0.005106
not-logged-in-41e9fe197dc7a2be8424    0.020657
not-logged-in-4cd99e50bbc7058bf4e8    0.000671
acapirala                             0.025403
rhondacard                            0.002664
seppe.ds                              0.016020
Lavadude                              0.013698
jordanrushworth                       0.010532
not-logged-in-4ea23f5fffc8b98635af    0.022740
Denny2507                             0.002661
pirxi                                 0.000881
ywan3529                              0.000612
swalla46     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())